<h1> <center> METRO MANILA: COMMODITY PRICE CHECKER </center> </h1>

<h1 style="background-color: #3cb371; padding: 20px;"> <center> <span style="color:white">
<br> INSTALLATION
</span> </center> </h1>

In [1]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pmdarima.arima.utils import ndiffs

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split

import itertools
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows', None)

<h1 style="background-color: #ffa500; padding: 20px;"> <center> <span style="color:white">
<br> COMMODITY PICKER
</span> </center> </h1>

In [3]:
# FUNCTIONS: CHOOSING A SPECIFIC COMMODITY TYPE
def rice_group():    
    rice_type = int(input('''CHOOSE A COMMODITY GROUP:
[1] REGULAR-MILLED RICE    [3] PREMIUM RICE
[2] WELL-MILLED RICE       [4] SPECIAL RICE
'''))

    match (rice_type):
        case 1: # REGULAR-MILLED RICE
            commodity = 'Regular-Milled Rice'
            csv_path = '../csv/rice/regular_milled_rice.csv'
        case 2: # WELL-MILLED RICE
            commodity = 'Well-Milled Rice'
            csv_path = '../csv/rice/well_milled_rice.csv'
        case 3: # PREMIUM RICE
            commodity = 'Premium Rice'
            csv_path = '../csv/rice/premium_rice.csv'
        case 4: # SPECIAL RICE
            commodity = 'Special Rice'
            csv_path = '../csv/rice/special_rice.csv'

    return commodity, csv_path
    
def meat_group():    
    meat_type = int(input('''CHOOSE A COMMODITY GROUP:
[1] BEEF BRISKET           [4] PORK BELLY
[2] BEEF RUMP              [5] PORK HAM
[3] WHOLE CHICKEN
'''))

    match (meat_type):
        case 1: # BEEF BRISKET 
            commodity = 'Beef Brisket'
            csv_path = '../csv/meat/beef_brisket.csv'
        case 2: # BEEF RUMP
            commodity = 'Beef Rump'
            csv_path = '../csv/meat/beef_rump.csv'
        case 3: # WHOLE CHICKEN
            commodity = 'Whole Chicken'
            csv_path = '../csv/meat/whole_chicken.csv'
        case 4: # PORK BELLY
            commodity = 'Pork Belly'
            csv_path = '../csv/meat/pork_belly.csv'
        case 5: # PORK HAM
            commodity = 'Pork Ham'
            csv_path = '../csv/meat/pork_kasim.csv'

    return commodity, csv_path
    
def fish_group():    
    fish_type = int(input('''CHOOSE A COMMODITY GROUP:
[1] ALUMAHAN               [3] GALUNGGONG
[2] BANGUS                 [4] TILAPIA
'''))

    match (fish_type):
        case 1: # ALUMAHAN
            commodity = 'Alumahan'
            csv_path = '../csv/fish/alumahan.csv'
        case 2: # BANGUS 
            commodity = 'Bangus'
            csv_path = '../csv/fish/bangus.csv'
        case 3: # GALUNGGONG
            commodity = 'Galunggong'
            csv_path = '../csv/fish/galunggong.csv'
        case 4: # TILAPIA
            commodity = 'Tilapia'
            csv_path = '../csv/fish/tilapia.csv'

    return commodity, csv_path

def fruits_group():   
    fruits_type = int(input('''CHOOSE A COMMODITY GROUP:
[1] BANANA (LAKATAN)       [3] MANGO
[2] CALAMANSI              [4] PAPAYA
'''))

    match (fruits_type):
        case 1: # BANANA (LAKATAN)
            commodity = 'Banana (Lakatan)'
            csv_path = '../csv/fruits/banana_lakatan.csv'
        case 2: # CALAMANSI
            commodity = 'Calamansi'
            csv_path = '../csv/fruits/calamansi.csv'
        case 3: # MANGO
            commodity = 'Mango'
            csv_path = '../csv/fruits/mango.csv'
        case 4: # PAPAYA
            commodity = 'Papaya'
            csv_path = '../csv/fruits/papaya.csv'

    return commodity, csv_path

def vegetable_group():   
    vegetable_type = int(input('''CHOOSE A COMMODITY GROUP:
[1] CABBAGE                [4] TOMATO
[2] CARROTS                [5] POTATO
[3] EGGPLANT               
'''))

    match (vegetable_type):
        case 1: # CABBAGE 
            commodity = 'Cabbage'
            csv_path = '../csv/vegetables/cabbage.csv'
        case 2: # CARROTS
            commodity = 'Carrots'
            csv_path = '../csv/vegetables/carrots.csv'
        case 3: # EGGPLANT
            commodity = 'Eggplant'
            csv_path = '../csv/vegetables/eggplant.csv'
        case 4: # TOMATO
            commodity = 'Tomato'
            csv_path = '../csv/vegetables/tomato.csv'
        case 5: # POTATO
            commodity = 'Potato'
            csv_path = '../csv/vegetables/white_potato.csv'

def spices_group():   
    spices_type = int(input('''CHOOSE A COMMODITY GROUP:
[1] GARLIC               [2] ONION
'''))

    match (spices_type):
        case 1: # GARLIC 
            commodity = 'Garlic'
            csv_path = '../csv/spices/garlic.csv'
        case 2: # RED ONION 
            commodity = 'Red Onion'
            csv_path = '../csv/spices/onion.csv'
            
    return commodity, csv_path

<h1 style="background-color: #54657e; padding: 20px;"> <center> <span style="color:white">
<br> ARIMA MODEL: PARAMETERS
</span> </center> </h1>

In [ ]:
# Define the GridSearch function
def grid_search_arima(df, column):
    """
    Perform grid search to find the best (p, d, q) parameters for ARIMA.

    Args:
    df (DataFrame): The input dataframe.
    column (str): The column name to analyze.
    p_values (range): Range of AR values.
    d_values (range): Range of differencing values.
    q_values (range): Range of MA values.

    Returns:
    tuple: Best (p, d, q) parameters and corresponding MSE.
    """
    
    # Define the range of parameters
    p_values = range(0, 10)  # AR order
    d_values = range(0, 3)  # Differencing order
    q_values = range(0, 10)  # MA order

    data = df[column].dropna()  # Ensure no missing values
    best_score, best_params = float("inf"), None
    for p, d, q in itertools.product(p_values, d_values, q_values):
        try:
            # Fit ARIMA model
            model = ARIMA(data, order=(p, d, q))
            results = model.fit()
            # Calculate mean squared error for the fitted model
            predictions = results.fittedvalues
            mse = mean_squared_error(data[d:], predictions[d:])
            if mse < best_score:
                best_score, best_params = mse, (p, d, q)
                
            print(f"p: {p}, d: {d}, q: {q}")
            print(f"MSE: {mse}")
        except Exception as e:
            continue
            
    return best_params, best_score

<h1 style="background-color: #c24b5a; padding: 20px;"> <center> <span style="color:white">
<br> ARIMA: TESTING MODEL
</span> </center> </h1>

In [5]:
# FUNCTION: Training and testing the model on the training and testing data
def training_model(commodity, df, p, d, q):
    # Store the dataframe in a separate variable
    X = df
    
    # Split the dataset into training (X_train) and testing (X_test) sets
    X_train, X_test = train_test_split(X, test_size=0.2, shuffle=False)
    
    print(f'Training Set: {X_train.shape} \n Testing Set: {X_test.shape}')

    # Train the ARIMA model with the high prices of the training set
    # Use the values obntained from the ACF and PACF Test
    model = ARIMA(X_train['High'], order=(p, d, q))
    model = model.fit()

    # Run the prediction
    start = len(X_test)
    end = len(X_train) + len(X_test) - 1
    predict = model.predict(start=start, end=end, typ='levels')
    predict.index = df.index[start:end+1]

    # Plot the predicted values, and the actual values from the Test set
    predict.plot(legend=True)
    X_test['High'].plot(legend=True)
    
    # Set labels and title
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Metro Manila: {commodity} Price Prediction')
    
    # Add a legend
    plt.legend()

<h1 style="background-color: #5f665a; padding: 20px;"> <center> <span style="color:white">
<br> ARIMA: FULL DATASET
</span> </center> </h1>

In [6]:
# FUNCTION: Training and testing the model on the full dataset
def full_model(commodity, df, p, d, q):
    # Re-train the model on the entire dataset, then predict future prices
    model = ARIMA(df['High'], order=(p, d, q))
    model = model.fit(method_kwargs={'warn_convergence': False})

    # For Future Dates
    index_future_dates = pd.date_range(start='30 08 2024', end='30 11 2024')
    
    # Run the prediction
    start = len(df)
    end = len(df) + len(index_future_dates) - 1
    predict = model.predict(start=start, end=end, typ='levels').rename('ARIMA')
    
    # Transform predict from Series to DataFrame
    predict_df = pd.DataFrame(predict)
    
    # Reset the index to turn it into a column
    predict_df = predict_df.reset_index(drop=True)
    
    # Add the future dates as a new column 'Index'
    predict_df['Index'] = index_future_dates

    # Reset the index of the column
    df = df.reset_index()
    df = df.rename(columns={'index': 'Date'})

    # Given timestamp
    start_date = last_row = df['Date'].iloc[-1]
    start_date = pd.Timestamp(start_date)
    
    # End date, which is 7 days after the start date
    end_date = start_date + pd.Timedelta(days=7)
    
    # Create a date range
    date_range = pd.date_range(start=start_date, end=end_date)
    
    predict_14 = pd.DataFrame()
    predict_14_price = []
    predict_14_days = []
    for day in range(7):
        filtered_df = predict_df[predict_df['Index'] == date_range[day+1]]
        predict_14_price.append(filtered_df['ARIMA'].values[0])
        predict_14_days.append(date_range[day+1])
        
    predict_14['Prediction'] = predict_14_price
    predict_14['Date'] = predict_14_days

    # Get the data from the original dataframe of the last 20 days
    last_14_days = df.tail(20)
    last_14_days = last_14_days.reset_index()

    # Get the full date range by concatenating the two dataframes
    full_date = pd.concat([last_14_days, predict_14])

    # Set the figure size of the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Ensure that 'Date' columns are in numeric format for polynomial fitting
    # If 'Date' is in datetime format, you need to convert it to ordinal numbers
    predict_14['Date'] = pd.to_datetime(predict_14['Date'], dayfirst=True)
    last_14_days['Date'] = pd.to_datetime(last_14_days['Date'], dayfirst=True)
    
    # Create the Line Plot: Curved
    sns.lineplot(x=last_14_days['Date'], y=last_14_days['High'], color='#003366', label='Predicted Price', alpha=0.5)
    sns.lineplot(x=predict_14['Date'], y=predict_14['Prediction'], color='#003300', label='Price')
    
    # Add vertical line at the last date of 'last_14_days'
    plt.axvline(last_14_days['Date'].iloc[-1], color="red", linestyle='--', label="Prediction Point")
    
    # Scatter plot for predicted values
    sns.scatterplot(data=predict_14, x=predict_14['Date'], y=predict_14['Prediction'], color='#003366', label='Predicted Price', s=40)
    
    # Set labels and title
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Metro Manila: {commodity} Price Prediction')
    
    # Add a legend
    plt.legend()
    
    # Set x-ticks to correspond to Date
    plt.xticks(full_date['Date'], fontsize=8, rotation=90)
    
    # Show the plot
    plt.show()

<h1 style="background-color: #00427c; padding: 20px;"> <center> <span style="color:white">
<br> GENERAL FUNCTION
</span> </center> </h1>

In [7]:
# FUNCTION: GENERAL
def gen_function():
    commodity_group = int(input('''COMMODITY GROUPS:
[1] RICE                   [4] FRUITS
[2] MEAT                   [5] VEGETABLES
[3] FISH                   [6] SPICES
Choose a commodity group: '''))

    # Rice
    match (commodity_group):
        case 1: # Rice
            commodity, csv_path = rice_group()
    
        case 2: # Meat
            commodity, csv_path = meat_group()
    
        case 3: # Fish
            commodity, csv_path = fish_group()
    
        case 4: # Fruits
            commodity, csv_path = fruits_group()
    
        case 5: # Vegetables
            commodity, csv_path = vegetable_group()
    
        case 6: # Spices
            commodity, csv_path = spices_group()

    print(f'You have chosen {commodity}.')

    df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)

    # Display plot of dataframe
    df['Low'].plot(figsize=(12,5))

    # Perform grid search for "High" column
    high_params, high_mse = grid_search_arima(df, "High")
    print(f"Optimal Parameters for 'High': {high_params}")
    print(f"Best MSE for 'High': {high_mse}")
    
    # Perform grid search for "Low" column
    low_params, low_mse = grid_search_arima(df, "Low")
    print(f"Optimal Parameters for 'Low': {low_params}")
    print(f"Best MSE for 'Low': {low_mse}")

    return commodity, df

In [8]:
def predict(commodity, df):
    p = int(input('Enter the value for p: '))
    d = int(input('Enter the value for d: '))
    q = int(input('Enter the value for q: '))

    execute_model = int(input('Do you want to proceed on training the model by:\n[1] Train-Test Data\n[2] Full Dataset\n[0] Exit'))
    match execute_model:
        case 1:
            training_model(commodity, df, p, d, q)
        case 2:
            full_model(commodity, df, p, d, q)
        case _:            
            print('Thank you for using the application.')

    return commodity, df, p, d, q

<h1 style="background-color: #71427c; padding: 20px;"> <center> <span style="color:white">
<br> MAIN
</span> </center> </h1>

In [ ]:
## MAIN: Run to choose commodity
commodity, df = gen_function()

COMMODITY GROUPS:
[1] RICE                   [4] FRUITS
[2] MEAT                   [5] VEGETABLES
[3] FISH                   [6] SPICES
Choose a commodity group:  2
CHOOSE A COMMODITY GROUP:
[1] BEEF BRISKET           [4] PORK BELLY
[2] BEEF RUMP              [5] PORK HAM
[3] WHOLE CHICKEN
 1


You have chosen Beef Brisket.


D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
D:\Programs\Python\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
D:\Programs\

In [ ]:
# MAIN: Run to execute prediction
predict(commodity, df)